## Lab Protocols <a id="TOC"></a>   
### Shantel A. Martinez |  Update 2018.08.30  
-------------  
**TABLE OF CONTENTS:**    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [Hormone Extraction](#horm_ext)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [LCMS Sequence Setup](#LCMSSeq)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [LCMS Run Protocol](#LCMSRun)   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [Measuring Dry Weight](#MeDryw)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [Lyophilizer/FreezeDrier](#Lyoph)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [Germination Assay](#GermAssay)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [DNA Extraction](#DNA_ext)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [RNA Extraction](#RNA_ext) 

### LCMS data analysis using Agilent OpenLab and R  
Protocol for data analysis of runs conducted in Columbia, Missouri (USDA-ARS in Oliver Lab) using the Agilent LC-MS (single MS). Instructions by Sven Nelson. Steps include calibration using a standards dilution series, peak calling, setting up the dilution factor and other settings for correct output from Agilent software, and downstream processing using R. R code for this analysis was written by Sven.  

**Required information:**  
A. Sample mass for each sample (filled tube after lyophilization - empty tube)  
B. Initial amount of IS (for each GA and for ABA) added per sample in ng.  
C. Final dilution and injection amount to determine dilution factor/multiplier.  

**Analysis**  
**Step 0:** (this time only) rename run files with correct sample names (correct my error).  
↳ Edit SAMPLE.XML <Name></Name> value for each sample in a run and delete *.B file at top level under run folder (or move it to a folder called “Trash” at the same level for backup). This method isn’t perfect, but names will change in Agilent software and naming for output file will change. The name of the sample in the output file will remain unchanged (but then you can
manually edit it). 

Note: If you close OpenLab CDS offline for some reason and it won’t reopen, giving you some error about the license server being down, specifically lmgrd (error -91, 121) then just restart Windows to solve this. (I tried to solve without restarting for a long time, but restarting was the only thing that worked.)  

**Step 1:** Setup report settings:  
`Report` `Specify Report…`  
Under Report settings:  
a. Select: “Use Intelligent Reporting”  
b. Set the report template to “Sample_Summary (4 decimal places)” (Sven custom template)  
c. Set the calibration template to “Calibration”  
d. When viewing reports, you may want to select “Screen,” but for file output select “File”  
e. Then Repo8rt file name should be “<Date> <SampleName>” and file type should be .XLS Note: Do not select .PDF and .XLS or it will only make a PDF for some reason  
f. Select: “Copy report to:” and select a location where you would like reports output. You will access these with R later.  

Under Quantitation settings:  
`Calc: ISTD`, Based on: Area, use multipliers etc√, Multiplier: 2.3333…, Compnd & ISTD amt then a dilution factor could be used to identify how many mL of the final sample were present and these will be multiplied. I need to double check this method because I don't use it anymore. Alternatively: ISTD amount is the final amount of ISTD for each compound, in ng. Then
multiplier is what that number needs to be multiplied by to reach the starting ng of ISTD per sample (Initial amount ISTD/sample). This is easier to calculate because you know what you put in and you don’t need to divide by the volume at any point.  

So in the case below, I started with 14 ng per sample (which was 7μl at the end of extraction) and injected 3μl (7/3 = 2.33…) giving an injected amount of 6 ng. This left enough for 2 injections.  
    
**Step 2:** Setting up calibration settings:  
`Calibration` `Calibration Table… `There may already be a calibration table from last time, but need a fresh calibration table.  
Useful info: Calibration table    
For IS samples: Amt stays constant (while changes for endogenous) Ref = No, ISTD = Yes   
For endogenous form samples: Amt change (while IS stays constant) Ref = No, ISTD = No  

First “#” is an id overall, last “#” should be the same for matching endogenous and IS pairs. Amt is ng (of IS or end) per inj. 

 5/24/2017 calibrations 7ng:2ng 
miscalculated 9/21 and 10/27 (not 8ng:2ng) 
9/21/2017 calibrations 0.8ng:0.2ng (actual) 
10/27/2017 calibrations 0.8ng:0.2ng (actual) 
GA12-Ald-IS was 1.6ng for 9/21 and 10/27 
Useful info: Calibration settings (see image →) 
For Default Calibration Curve: 
Type = Linear (we expect a linear correlation) 
Origin = Force (we expect this goes through 0,0) 
Weight = Equal (each value has equal weight) 
Other settings generally weren’t changed. 
Next , create a new calibration table: 

**OR**  
`Calibration` `Recalibrate…` `replace` and adjust the Amt in the table (Recalibrate doesn’t work if RTs have changed, so easier to just make a new calibration table.)  

SO  
1) `Calibration` `New Calibration Table…`   
- Automatic Setup = √  
- Level 1 (will increase this for each subsequent run)  
- Default Amount: 0.8 (This would be the ng amount for IS. Will need to adjust this for end later.)  
- Calculate Signals Separately = √  
- Overwrite existing calibration table? = YES  

2) Data for all peaks will appear (both signals) from the currently selected run. You need to do the following:   
- Add compound names in “Compound” column (order is: GA8, GA29, GA1, ABA, GA19, GA20, GA53, GA12, GA12-Aldehyde)  
- Correct “Amt” column for endogenous peaks (also GA12-Ald-IS was 1.6 this time)  
- Change ISTD column to “Yes” for all IS rows (must do this before adjusting last column)  
- Adjust last column “#” so that IS and end pairs have the same number    
3) Do this first! Set integration start & stop for all peaks. Make it broad to cover the entire peak  

To adjust start and stop for peak integration:  
- First go to Calibration > Calibration Table Options… > Identification Details (Normally you are on “Overview”)  
- Adjust the From and To for your peaks.  
- Also adjust RT values where it appears off.  
- Def will become unchecked. Leave it unchecked.  
- In my experience, later peaks (GA53, GA12, and GA12-Ald) were automatically given wider integration areas. Possibly because later peaks are more likely to be wider? These can be left unchanged if already wide. (if this causes issues of non-specificity, come back and change the width.)  
- Switch back to “Overview” at the end.  

`Calibration` `Calibration Table Options…` `Overview` or use the Calib toolbar drop-down
4) Now open the next run from the navigation table (Level = 2).
- Calibration > Add Level…
- Level = 2, Default Amount: 0.8
5) If any peaks aren’t added, it probably means you need to readjust the From and To again.
- I tried adding them using Calibration > Add Peak…, but it didn’t produce the desired
results
- Easiest was to adjust the peak From and To, then delete all rows for this new level and add
a new level again (with the same number)
- It is possible that if I had just adjusted the From and To first, then done Add Peak (or
Insert… and chosen peak) that this would have worked without deleting things.
6) Once all level 2 peaks are present correct the amounts in the amount column.
- No need to add compound name, lvl 2 peak shows up under lvl 1 for the same compound
- Correct the Amt column amounts
- No need to adjust the last “#” column or any other columns
7) Now repeat this for the remaining levels (remaining calibration runs).
- Repeat #4-6 until all standards have been added
- Click OK.
8) Finally, go through each compound (end and IS) and click on the first row to check the curve.
- The curve should appear to the right with an equation above and R2 below (Correlation)
- For endogenous compounds, check that the Correlation is at least 0.99
- For IS you do not have multiple dilutions, but check that the points are generally in the
same vertical (y-axis) location.
- If either of these is off, check that you haven’t just input a value wrong in the Amt column.
- If they are still off and you believe it is due to the real peaks, you may need to redo the run
with standards (maybe you diluted wrong… maybe you are too close to the detection limit
or too high so that you are not in the linear range).
9) When done, click “OK” and correct any errors produced. You are now calibrated!

Go back and Check Quantitation settings under Specify report
(ISTD Amounts should be automatically set now, but just check)
Save the method.
Recalculate > With method… (may want to set integration first)
↳ Select method (I also selected report, but not sure if it matters)
↳ Select “None (no report)” or it will waste time generating reports.
↳ If you order runs by time, it will auto step through all runs in
order.
Next, Adjust Integration ON/OFF in Integration settings
↳ This is useful for automatic peak integration, but in cases where
peaks are very difficult to catch automatically, it it not worth the time of setting this up. Manual
integration will not be prevented in “OFF” areas. So just do manual integration by hand if it looks
like that is going to be faster than automated integration.
↳ Adjust integration (manual integration) of all peaks for the
dilutions series
↳ Calibration > Specify Report
Set it back to export to File and xls. Specify location.
4) Manual integration of all sample peaks (if auto integration
looks good, leave it be)
5) Export reports (xls deposited in folder set above)
↳ If reports fails to export when you hit print (nothing happens), just
restart windows and restart the software, then try again. This
generally solves the problem. (probably caused by virtual machine pausing, but unsure why.)
6) Use R to pull out info from reports and do plotting or other analysis as necessary.   

--------------
<div style="text-align: right"> [TOC](#TOC) </div>  

### LCMS Peak Calling    

Load `Method` `_SIM4A_SHANTEL` : this will also include the calibration settings already including while creating this method.   
Open Data file that you want to analyze: Left Menu  
Double `Click` on sample file  
View the `Integration` menu.  
Delete auto-integration lines.   
Manual integrate calls (use zoom when necessary)  
Double `Click` next sample file, a menu will pop up requesting to save changes, click `Yes`  
Repeat manual integration calls    

-------------- 


### Extraction for Hormone Measurement (USDA-ARS MU 2017) <a id="horm_ext"></a>     
*Author: Sven C. Nelson, PhD | Comments: Shantel A. Martinez, PhD*  
*This is the extraction of samples for ABA and GA (and GA precursor) measurement in Columbia, Missouri (USDA-ARS in Oliver Lab) using LCMS. Based on the protocol from RIKEN 2013, but with substitutions for tools that we do not have here. The 2016 version of this protocol used 15 mL tubes, but it has now been further modified to use **5 mL polypropylene round-bottomed tubes (Falcon, product: 352063)** for most steps. This allows speed-vac steps to accommodate 40 tubes at once with our new rotor for 5 mL tubes. For larger samples, refer to 15 mL or 50 mL tube protocols. (Max 400 mg sample : 50 mL tubes.)*   

<div style="text-align: right">  5 mL tubes used: 1 </div>  
<span style="color:#6E8B3D">**START DAY 0**</span>   
**1. Measure the weight of the freeze-dried samples in 5 mL tube.** (desired: 50-100 mg DW)  
>a. Do this beforehand. Pre-weigh empty tubes before adding ground tissue and then weigh again after freeze drying. Subtract the first from the second to get the total dry weight. Be sure to use the sensitive scale and record all 4 digits after the decimal for your pre-weights.   

<div style="text-align: right">  5 mL tubes used: 0 </div> 
<span style="color:#6E8B3D">**START DAY 1**</span>  
<span style="color:#838B8B">**&#8615; ON BENCH OR IN HOOD**</span>  
**2. Add 2 mL of the 1st extraction solution to each samples**   
>a. For all extraction solutions, use HPLC quality H2O. Don’t store extraction solution as it contains internal standards (ISs) which can degrade over time, make fresh.  
>b. 1st extraction solution: 80% Acetone containing 1% AcOH and IS-MIX solution  
>c. Add IS (GA and ABA internal standards) to extraction solution “master mix” so that each sample will get the desired final ng per sample. **Record the per-sample ng of IS added.**  
>d. **How much IS-MIX to add?** Decide based on amount of endogenous GA/ABA expected
and peaks within the linear range of detection in test LCMS runs of ISs.  
>e. **Example:** If 7 μl final volume (per sample) and injecting 3 μl. If you desire 24 ng/injection, then there should be 56 ng/sample (24 × 7/3). However we are adding 2mL per samples, therefore 56/2 = 28 ng/1mL.     

.

|**1st Extraction Solution**|Chemical|Concntration|40mL|45mL|unit|
|:---|:---|:---|:---|:----|:----|
||Acetone|80%|32|36|mL|
||AcOH|1%|400|450|μL|
||Internal Standard Mix|24ng/inj & 48ng/inj|291.2|326.8|μL|
||H2O|-|7.6|8.55|mL|   
  
|A|B|  
|:---|:---|  
|![](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/ExHormoneISchecklist.jpg?raw=true) | ![](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/ExHormoneIScalc.jpg?raw=true)|  
  
**3. After mixing by vortex, incubate the tube for 6 hours (or ON) at 4ºC.** (1st incubation)  
<div style="text-align: right">  5 mL tubes used: 1 </div> 
<span style="color:#6E8B3D">**START DAY 2**</span>    
**4. After 6 hours, remove caps, centrifuge samples at 4000 rpm (3399 g) for 10 min.**  
>a. Large centrifuge, max 5100 rpm. Settings: S.51 and set PLA/BUC to BUC (buckets).  
>b. While spinning 1) label new tubes (tube set #2) 1-40 if you havent already done so, 2) make 2nd Extraction solution (see step 6), and 3) Turn on speedvac with vapor trap (it takes 90min to reach temperature; see step 10).  

**5. Collect the supernatant in labeled 5 mL tubes (save the supernatent tube set #2 until step 9).**  
>a. Use a 1mL pipette with fresh tips very time.  
>b. The supernatent tube set #2 can be left on the bench (or in a drawer to keep IS from degrading in the dark) or at 4C if left for a longer period of time.  

**6. Add 2 mL of the 2nd extraction solution to the pellet (1st tube set) of each sample.**  with repeat pipettor, reusable tips undr the hood, 1mL x2.  
>a. 2nd extraction solution: 80% Acetone containing 1% AcOH. Make this with the units on the falcon tube).  
>b. Make 3x 50mL falcon tubes, the solution can sit over time. 
>c. re-cap tubes for next step.  

|**2nd Extraction Solution**|Chemical|Concntration|50mL|unit|
|:---|:---|:---|:---|:----|
||Acetone|80%|40|mL|
||AcOH|1%|500|μL|
||H2O|-|fill to 50mL|mL|  

**7. After mixing by vortex, incubate the tubes for 10 min at 4ºC. (2nd incubation)** Votex time: count to 10   
**8. Centrifuge samples without caps at 4000 rpm (3399 g) for 10 min.**  
**9. Collect the supernatant (tube set #1) and add to the same 5 mL tube (tube set #2) from step 5 (discard pellet)**  
>a. Use a 1mL pipette with fresh tips very time.  

**10. Evaporate extraction solution using speedvac in lab (new 40 × 5 mL tube rotor)**  
>a. 43-45°C, vacuum level 20.0, heat time 4hrs, time required: ≈3 hours (it’s ok if ≈0.5 mL or less remaining)
>b. To turn speed vac on: switch speedvac switch on, vapor trap swtich on, pump switch on (if it didnt turn on with the vapor trap)  
>c. Change the rotor 0-12 that fits 40 5mL falcon tubes.  
>d. The vapor trap needs to reach the temperature -104C (takes ~90min) 

<div style="text-align: center"><span style="color:#CD5C5C">LUNCH</span></div>    
<span style="color:#838B8B">**&#8615; IN HOOD**</span>  
**11. Add 2 mL of Hexane to the dried down sample.**  
> We are using the Hexane to pull out un-wanted junk  

**12. Add 2 mL of 50% Acetonitrile (MeCN) and vortex.**  
>a. Solution will form two layers or can (but not necessary) be centrifuged at 8000 rpm for 5 min  

**13. (3×) Remove the upper layer (Hexane) and discard, add 2 mL Hexane.**  
>a. 3× = do this a total of 3 times, check the first box and repeat two more times: ☐ ☐ ☐  
>b. *Do not add Hexane on the 3rd time and move on to step 14*  
>c. In layman's terms: 1) remove Hexane layer, discard, 2) add Hexane, vortex, remove Hexane layer, discard and 3) add Hexane, vortex, remove Hexane layer, discard. Proceed to Step 14   

**14. Evaporate extraction solution using speedvac (no heat, run overnight OR 43°C, ≈3-4 hr).**  
>a. For the next day: Make PVP solution (50/50 of PVP and HPLC-grade water) if needed.   

PREP FOR TOMORROW: Make 0.5M K2HPO4 soln (see step 15 notes); Label 2 sets of tubes; Make 1% AcOHaq     

<div style="text-align: right">  5 mL tubes used: 2 </div> 
<span style="color:#6E8B3D">**START DAY 3**</span>  
<span style="color:#838B8B">**&#8615; ON BENCH**</span>  
**15. Add 0.5 mL 0.5M K2HPO4 to the dried down sample, vortex, and allow to dissolve.**
>a. Add 4.355 g to total of 50 mL of water; MW = 174.18   
>b. While waiting for samples to dissolve, prepare and condition the PVP columns (step 16)  

**16. Conditioning the PVP column (PVP = Poly(vinylpolypyrrolidone), ex: Sigma prod. 77627)**  
>a. Fill Varian Reservoir Frits 1 mL columns (Agilent 12131013) to 1/2 volume with PVP  
>>i. To do this, add 1 mL of 50/50 mix of PVP and HPLC-grade water to column.  
>>ii. Use a syringe with column adapter to gently push through some water to start the gravity-flow of liquid through the column  
>>iii. Check pH of flow through with lithmus/pH paper, it is probably about pH 7  
>>iv. Add ≈500μl each time of 0.5M K2HPO4 up to 2 mL (4 times) or until the pH gets to about pH8~9 (Check flow-through with litmus/pH paper) ☐ ☐ ☐ ☐  

**17. <span style="color:#4F94CD">Collect</span> flow-through: Once dissolved load the 0.5 mL sample in the PVP column, add another 0.5 mL 0.5M K2HPO4 to the sample tube and add this to the column. ☐ ☐**  
>a. (optional) If sample is dirty, can first pass through and empty 3mL Varian Reservoir Frits column (0.5 mL × 2) to remove larger debris, then add that 1 mL to the PVP columns**  
>b. lift rack to see if liquid went through & tap to let last drop fall.   
>c. PVP/column can be discarded in regular trash  

**18. <span style="color:#4F94CD">Collect</span> flow-through: add 2 mL 0.5M K2HPO4 (4 × 500μl) to elute. ☐ ☐ ☐ ☐**  
>a. Total volume of flow-through is 3 mL  

<div style="text-align: center"><span style="color:#CD5C5C">LUNCH  
*suggested stopping point, but any break above works too. I tend to eat during step 18 while I wait*</span></div>  
<span style="color:#838B8B">**&#8615; IN HOOD**</span>  
Turn on speed vac + vapor trap (90min prior to step 24)    
**19. Add 6N HCl 100 - 300 μl to the sample until the pH is 2 - 3, check by litmus/pH paper.**    
>a. add 100uL to all, spin uncapped, check multiple tube pH's. 
>b. add 100uL to the #1 tube, check pH, add 100uL more if you need more to get 2-3 pH.  
>c. then add 200uL to all of the tubes (or 100uL if you only needed 100uL more instead of 200uL).  
>d. spin uncapped, check pH, recap, and set in the dark drawer, and procees to step 20 

**20. Conditioning the 3 mL HLB column (Oasis HLB 3cc Vac Cartridge, Waters WAT094226)**  
>a. Add 3 mL of MeCN and discard flow-through  
>b. Then 3 mL of MeOH and discard flow-through  
>c. Then 3 mL of 1% AcOHaq and discard flow-through  
>> Columns are snug in the column holder, dont need to push all the way down.  
>> Constantly remove the waste solution into properly labeled `Waste Container`  

|Chemical|Conc.|Amount|unit|
|:--|:--|:--|:--|
|AcOH|1%|1|mL|
|LCMS H2O|-|99|mL|  


**21. <span style="color:#4F94CD">Discard</span> flow-through: Load the extracts to HLB (ABA/GA is retained in column).**  
**22. <span style="color:#4F94CD">Discard</span> flow-through: Add 2 mL of 1% AcOHaq twice (2× wash step). ☐ ☐**    
>a. Near the end, you can use the syringe to push the column solution all the way through, to get that extra last bit out. However push slowly and steady.   

**23. <span style="color:#4F94CD">Collect</span> flow-through: Move columns above new 5 mL tubes, add 2 mL 80% MeCN containing 1% AcOH twice (2× elution step).** ☐ ☐   
> 4mL/sample * 40 samples = 160mL 

|Chemical|Concntration|160mL|unit|
|:---|:---|:---|:----|
|MeCH|80%|128|mL|
|AcOH|1%|1.6|mL|
|H2O|-|30.4|mL|  

**24. Evaporate extraction solution using speedvac (43°C, run overnight OR 43°C, ≈4-5 hr).** no liquid left    

<div style="text-align: right">  5 mL tubes used: 2 </div>   
<span style="color:#6E8B3D">**START DAY 4**</span>    
<span style="color:#838B8B">**&#8615; IN HOOD**</span>  
**25. Add 0.5mL MeOH to each of the dried down samples, pellet will dissolve easily, vortex briefly to mix.**    
<div style="text-align: center"><span style="color:#CD5C5C">*DO NOT ALOW COLUMNS TODAY TO DRY OUT*</span></div>  
**26. Conditioning the 1 mL DEA columns (Bond Elut DEA cartridge, Agilent 14102016)**   
>a. Add 1 mL MeOH and discard flow-through  

**27. <span style="color:#4F94CD">Discard</span> flow-through: Load the extracts into the DEA columns (ABA/GA is retained).**  
**28. <span style="color:#4F94CD">Discard</span> flow-through: Add 1 mL MeOH three times (3× wash).** ☐ ☐ ☐   
**29. <span style="color:#4F94CD">Collect</span> flow-through: Move columns above new 5 mL tubes: Add 1 mL MeOH containing 0.5% AcOH three times (3× elution step).** ☐ ☐ ☐    

|Chemical|Concntration|150mL|unit|
|:---|:---|:---|:----|
|MeOH|-|150|mL|
|AcOH|0.5%|750|μL|

**30. Evaporate extraction solution using speedvac (43°C, ≈2 hr).**  
>a. Place at 4°C if dry before lunch.  

<div style="text-align: center"><span style="color:#CD5C5C">LUNCH</span></div>    
**31. Add 0.5 mL of 50% Chloroform 50% ethylacetate containing 0.1% AcOH to the dried down samples, vortex briefly.**  
**32. Conditioning 1 mL Sep-Pac Silica columns (Sep-Pac Silica 1cc, Waters WAT023595)**  
>a. Add 3 × 1 mL of 50% Chloroform 50% ethylacetate 0.1% AcOH (discard) ☐ ☐ ☐  

|Chemical|Concntration|440mL|unit|
|:---|:---|:---|:----|
|Chloroform|50%|220|mL|
|Ethylacetate|50%|220|mL|
|AcOH|0.1%|440|μL|

**33. <span style="color:#4F94CD">Collect</span> flow-through: Load extracts to Silica and collect in pre-labeled 5 mL tubes.**    
**34. <span style="color:#4F94CD">Collect</span> flow-through: Add 1 mL 50% Chloroform 50% ethylacetate containing 0.1% AcOH twice (2× elution step; total 2.5 mL in tube)** ☐ ☐   
**35. Evaporate extraction solution using speedvac (43°C, ≈1 hr).**    
>Can prep the LC/MS vials in for step 36 

<span style="color:#CD5C5C">*A time you can store @ RT for 1-2hr OR @4C >2hrs*</span>   

**36. Resuspend in 200μl MeOH, transfer to LC/MS vial insert, dry by speedvac (43°C, 30 m).**    
> For steps 36 and 37, you can use a speedvac without a vapor trap.  

**37. Repeat step 36: Add 200μl MeOH, transfer from 5 mL tube to vial insert, and dry down.**  
>a. Store in this form and resuspend for run.  

**38. For small injection sizes (eg 3μl per inj), resuspend in X μl of 99% MeCN 1% AcOH.**  
(X = inject vol × # of inj + extra; ex: 7μl; higher MeCN is needed for GA12/12-Ald peaks)  
For larger injections (eg 20μl per inj), resuspend in X μl of 15% MeCN 1% AcOHaq.  

PROCEED TO RUNNING SAMPLES ON LCMS

--------------
[TOC](#TOC)

### LCMS Sequence Parameters  <a id="LCMSSeq"></a> 
#### Example Parameters to keep record of:   
**Run Pameters**

|Folder|FileName||||Method|Injection|
|:--|:--|:--|:--|:--|:--|:--|
|NameDate|Date_01_1||||SV_PHENYL150X3MM3UM_ABA_GA_SIM4_Shantel.M|3uL|

**Sample Sequence**

|Tube|pos|reps|sample|
|:--|:--|:--|:--|
|ACN|91|1|MeCN|
|43|51|3|sample_001|
|42|52|3|sample_002|
|41|53|3|sample_003|
|ACN|91|2|MeCN|


**Updating Software Sequence Info**  
1. `Sequence` `Sequence Parameters` Subdirectory: `NameDate` Click Prefix cell (will ask to create, `yes`). Prefix should be the file name `Date_01_` and the counter will start with `001`  
Check post seq command: `STANDBY` For a single run. If I wanted to run multiple runs, and whaat to run right after one another. You can change WHILE your sequence is running. Uncheck it if I dont want the LCMS to go to standby.  `OK`  
2. `Sequence` `Sequence Table`  Click a cell and you can edit the cell. Input the pos into `vial` column. Highlight a row to cut it out. Insert to add more rows. Edit `Sample Name`, `Inj/Vial`, and `Method Name` needs to be my prefered method (all samples HAVE TO BE the same method). Once done, click `OK`  
>a. This can be done ahead of time during the extraction method.  

3. Save Sequence Template: `Sequence` `Save Sequence Template As...` Rename File Name `Date_Hormone_User_Set.S` `OK` (If you are re-saving it, click `Yes` to over-write the old file.   
4. Once the dilution samples are dried down, re-suspend to  7uL in 1% AcOH in MeCN (Assuming your sample is to be diluted to a final volume of 7uL, this may change on your preference).  
5. Place the samples in the LCMS tray and record the position in the `sample sequence table` above. 
6. Double check `Sequence Parameters`, and `Sequence Table`

--------------
<div style="text-align: right"> [TOC](#TOC) </div>

### LCMS Machine Run & Configuration <a id="LCMSRun"></a>  
The basis of the LC : sample/hormone will flow based on both the chemistry in the column AND the size.  
And the change of ratios of the solvents, you can push/pull out as slow or fast. Because if you have a low % solv B, things come out slower, but close peaks together seperate (more resolution of peaks close). But if you have a high % solvent B, things will come out faster. The same things with the disposal columns, but we arent keeping all, getting rid of things, this time we what to spread the hormones out, so when it gets shot out to the mass spec, it reads the masses.  

**1. LCMS Chemicals**  
First, Do we have enough solvents to run the LCMS
*Use HPLC graduate cylinder (in the hood). Rinse the cylinder with HPLC grade water before use*
When switching jars while on, turn the tube knob to waste   

||water + 0.1% Acetic Acid| Acetonitrile + 0.1% Acetic Acid|||
|:---|:---|:---|:---|:---|
|total|1700mL|1700mL|||
|Acetic Acid|1.7mL|1.7mL|||
|H2O/MeCN|Fill to total|Fill to total|||
|||Must mix very well or it will effect your results|||

Turn knob back to column.  
***MOST IMPORTANT THING TO REMEMBER WHEN CHANGING SOLVENTS***
When/if you change the solvents, you need to tel lthe computer how much is in the bottles. The machine will stop the run before it gets to 0mL.  
- Quant. Pump window click on the bottle letter. Ex: A change to 2.0 liters (full) with a max of 2.0 liters. 
- When you click, you can change all of the final volumes in the pop-up window.  
- Click Ok. Then the green liquid levels in the graphic will change. 

| A                                                            | B    | C    | D    |
| ------------------------------------------------------------ | ---- | ---- | ---- |
| ![LCMS1](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/LCMS1.jpg?raw=true)      | ![LCMS2](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/LCMS2.jpg?raw=true)     | ![LCMS3](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/LCMS3.jpg?raw=true)     |![LCMS4](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/LCMS4.jpg?raw=true)|  

<span style="color:#CD5C5C">**2. Turn on the pump 30min - 1hr before starting.**</span> 
- Start with turning on the machine with a lower pressure (0.5) `On`
- Once the temperature has reach around 60C, then change the pressure back to Method pressure: 1.0 (mL/min)

**3. G6100 (online)**   
>Sampler references black needle that will go from tube to tube. Idle box green (on)   
>Quant. Pump represents the A, B, C, D large chemicals.  The clear tubes have letter labels. Only ever replace these one at a time, so there is no chance at switching tubes in the wrong liquid. Stanby box (grey offline)    
>Column Comp: represents the colume, and there are two shown on the screen because you can have 2 seperate columns and adjust the temp. Displays temp. Not ready (yellow)  
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Watch the pressure to make sure its not  
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; too low (leak)  
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; too high at red bar (400mL/min)  
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If it gets close to 500 it can handle it, but that might be a sign there is a clog or something thats not too normal. Too much pressure you can develope a leak, or cause issue with other tubes. 
>MS: is the quadrapole shown for the mass spec (6120 Quadrapole MS)   
>Signal 1 plot is the sample endogenous form, signal 2 is the IS form.  
  
Every time you run the LCMS, make sure the `Method File icon` has the correct method name  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SV_PHENYL150X3MM3UM_ABA_GA_SIM4_SHANTEL.M  (sven_columntype_hormones_methodversionnumber_user.M)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; the methos `scan` will look at anything with all masses (so anything will show up, more noise that I dont want).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `sim` allows us to specify the molecular weight of what we are looking for (i.e., ABA) and then the mass spec will only scan at that MW. So you can do this on all the hormones you want to look at.    
      
4. Specify or change a `Run Method` (*in our case, we exluded GA19*)   
`Method` `save as` name the method in the folder `c:\chem32\1\methods`  
`Edit Entire Method` Leave all checked `OK` desciption of method you can edit but is not necessary `OK` ALS `OK`   
We wont change the Solv A and B ratios, Keep the fow rate at 1.000 mL/min. `OK` `OK`   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Temperature fluctuations can effect peaks, so he keeps a LEFT high temp at 60C.  And left and right are combined, so they both are the same.  
Set Up MSD Signal :Setting up the masses and the retention times we want the MS to start looking for our specific mass.     
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; general tune file: `atunes.tun` ALWAYS put it on this one, no matter what anyone else tells you. It points to the most recent tune file.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Mode `SIM` keep that mode   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Polarity, ABA/GA are `negative`, if you dont know run one with negative and then again with positive to see where your peaks show best. `OK` `OK`    
>If you need to edit the hormones, this mene is where you edit.  
>GA19 Group4 `Select` `cut` both lines   
>Change Group 6 retention times from 5.6 to 5.5  And do the same for the IS GA19 samples   
>The retention times for the Internal Standard or "endogenous" standards are differeent than when you run the samples.  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Sample Purity: looking to M- no adduct (mass) and M-H (M - 1) in the negative tab since we are looking `OK` `OK` `OK`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Specify Report: G6100 `Calssic Reporting` check `Screen`uncheck `File` and the Quantitation setting is set to certian numbers based on the injection amount we did (24 and 48).  And now everytime it runs, it should pop up on the screen. `OK` `OK` `OK` `OK` `OK` `OK`  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; `Method` `Save Method` MUST BE DONE now, or it will not run with this new updated method, it will run with the old method. Comment if you would like `OK`    

5. Set up Sequence Parameters and Tables (see [instructions here](#LCMSSeq)).  
6. Press the giant `ON` button on the screen, when you hear a sound, thats the pump working. MS is still offline, when we actually start a run, the MS is online. and the temperatures are coming up to temperature.  
> Make sure the pump setting is back to 1.000 mL/min  

7. *Turn `ON` the MS* 
Eveything should turn green and be ready. Once everything is ready, The top left tab will be green `ready`  
8. Double check `Sequqence Parameters`, and `Sequence Table`,  `Method`, and then re- `Save Method` just in case. 
9. To start the run: `Run Control` `Run Sequence` **NOT RUN METHOD**  
>a. If you ever need it to stop, you can press `Pause Sequence` then once it has paused after the current sample, THEN you can press`Stop Run/Inj...` and it will stop at the end of this sample (or the one following) but never in the middle of a sample. 

**General LCMS Notes**
Solvent B will go up in volume during the run, but Solvent A will constantly output the same volume, so solvent A always runs out quicker.   
`Documents` `Shantel`  `screenshot_logs`     
Label the files `date_columnname_hormone`  
HP Computer has no internet, since the software needs to be run on window 7 and not upgraded (and USDA has a policy that all computers with a network must have windows 10)    


![](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/LCMS_StandardPeaks.JPG?raw=true)


--------------
<div style="text-align: right"> [TOC](#TOC) </div>

<a id="MeDryw"></a>   
**Measuring Dry Weight for Hormone Extraction**   
**Step 1:** Label each tube with an idenifing number. 
> My labeling system is just the numbers in the 1000's, and all of he tube information is found in a google worksheet.  
> For the hormone extaction protocol, 5 mL polypropylene round-bottomed tubes (Falcon, product: 352063) is needed for downstream speed vac rotor size.  

EXAMPLE DATASHEET SET-UP
![](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/DatasheetEx.JPG?raw=true)

**Step 2:** Place tube holder (Below **Image A**) on the small balance.  
**Step 3:** Close the balance door. Wait 20 seconds for the tube holder to steady. Tare the tube holder. 
> *From this point on, always be patient. The precise weight is more important than the time it takes to tare, wait, weigh, wait, etc.*  

**Step 4:** Add the empty tube plus cap into the designated tube holding position. Close the balance door. Wait until the balance read stops on one value for 15+ seconds. 
> Changing where a tube is measured from tube to tube will result in inconsistent values, increasing the measuring error. So keep the position of the tube exactly the same for each tube, and preferably in the exact middle of the balance.  

**Step 5:** Record the empty weight value into the datasheet column "weight_empty". Record all 4 digits past zero. Repeat steps 3-5 for each tube.  
> The sample masses are key to determining the final concentration, so use the most sensitive scale that you have and record all 4 digits past the decimal.  (Ex. 7.0035g - 6.0125g = 0.091g) This information is used in the final data analysis to determine ng/g dry weight of the hormone in your samples.

**Step 6:** Prep samples per experimental conditions   
**Step 7:** Flash freeze samples in liquid N   
**Step 8:** Grind in liquid N. Make sure tissue is well ground. Place in -80C for a minimum 2 hours.  
>For dry seed tissue, add a few drops of water into the liquid N with the seeds before grinding to help break up the tissue.  

**Step 9:** Separate the lids and cover the tubes with parafilm that have poked holes from a tack.  
**Step 10:** [Lyophilize](#Lyoph) ground sample for 48hrs (Below **Image B**).  
**Step 11:** Replace tube lids with original lids (Below **Image C**). Keep track of which lid goes with which tube since each lid has a slightly different mass    
**Step 12:** Weigh the tube with lid now containing the lyophilized sample as in **Steps 3-5** above. Record the sample weight value into the datasheet column "weight_sample". With the "weight_sample" minus the "weight_empty", we can get the actual mass of the lyophilized sample (column “dry_weight”).   
**Step 13:** Parafilm the tubes tightly to prevent any sample from escaping and to prevent moisture entering during shipping.   
**Step 14:** Store and ship the tubes wrapped in aluminum foil to prevent any hormone breakdown due to light. You can store and ship these samples at room temperature.  

A | B | C
:--- | :--- | :--- 
![bal](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/DryWeightBal2.jpg?raw=true)  |![lyph](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/Lyophilizer.jpg?raw=true) | ![lyph](https://github.com/shantel-martinez/Lab_Resources/blob/master/example_img/EmbryoTubes.jpg?raw=true)   

--------------
<div style="text-align: right"> [TOC](#TOC) </div>  

### Lyophilizer  <a id="Lyoph"></a>      
**Set-up/Run**  
**Step 1.** Check the drain. Pull the tubes plug and drain the excess water into a tub  
>If the lyophilizer is in heavy use, it will have a lot of built up moisture.  

**Step 2.** Turn the machine on. On/Off switch is on the right side  
**Step 3.** Turn on the refridgerator 
> There are three main buttons: Two of them are for cooling. Auto and Man. Use manual so that the vacuum doesnt automatically turn on once cooled.  
> Wait for the temperature to come down to -40C - -50C (only takes about 10 min)  
> Warning: the plastic dust cap will fly off, you didnt break anything. It just keeps dust out for long term storage.      

**Step 4.** Add your samples into the chamber and close the chamber. Double check valves are closed. (Notch on top)  
**Step 5.** Turn on vacuum  
**Step 6.** In a few minutes, double check to see if te lyophilizer is pulling vacuum. 
> Note: It won't reach full green vacuum, because James has it set that way. The second to last light will blink.   

**Turn-off**  
**Step 1.** Break the vacuumm slowly by opening a valve   
**Step 2.** Turn off the vacuum pump button   
> If you know your samples are finished, you may take them out now and proceed with shutting down lyophilizer.  
> If the samples are not fully dried, repeat steps 4-6 above for additional hours of freeze drying.  

**Step 3.** Turn off the refridgerator button   
**Step 4.** Turn unit off switch   
> Remove the chamber and let thaw.  
> Wait until fully thawed before drainin the excess water.  

### Germination Assay <a id="GermAssay"></a>    
**Seed Sterilization**  
**Step 1.** Count out number of seeds (dry) and place the seeds in 50ml conical tubes
>a.	One line per tube, 3 reps of 10 = 30 seeds per treatment  
>b.	Treatments may include 2uM ABA, 5uM ABA, 5mM MES (No Hormone), or 10uM GA   

**Step2.** 	Add sterilization solution 5-10ml above dry seed line (not rocket science). Tightly screw lid.  
**Step3.** 	Shake to mix every 5 min for 15 min.  
**Step4.** 	Use the vacuum with a tip to suck out the solution (It’s hard to get out all solution, but try to get as much as you can)  
**Step5.** 	Add autoclaved dH2O 5-10ml above seed line (to dilute and wash out the sterilization solution)  
**Step6.** 	Shake every few minutes for 7min  
**Step7.** 	Vacuum off dH2O and repeat steps 5 & 6 twice (a total of 3 rinses)  
**Step8.** 	Use the vacuum to suck off autoclaved dH2O  
   
**Plating seeds**  
**Step1.** 	Use sterile petri plates and sterile filter paper; insert one blue filter paper into each plate  
**Step2.** 	 Label the top of each plate according to solution concentration, type of seed being plated, weeks after-ripened, and date plated  
**Step3.** 	Make hormone/treatment solutions and add 6ml into each plate  
> 6 mL per plate | Calculate 6.25 mL x Number of Plates  

**Step4.** 	Use small forceps to remove bubbles underneath filter paper (make sure the filter paper is flat on the bottom, may need to fold on the sides to fit)  
**Step5.** 	 Use small forceps to place each sterile seed onto the plate (embryo facing left, seed fold facing down)  
**Step6.** 	Plate 3 sets of 10 onto one plate  
**Step7.** 	Parafilm the top and bottom of the plate together in order to keep the moisture in  
**Step8.** 	Wrap 3-4 plates together in foil to exclude light. (shiny side in; dull side out)  
 
**Germination Screen**  
**Step1.** 	 Once plated, place in the 20-22°C incubator  
**Step2.** 	Score every 24 hours for 5 days after plated and incubated  
**Step3.** 	Day 1: mark on top of plate / for germinated seeds and X for 3-point seeds  
>a. Record both germinated seeds and 3-point seeds for each day  

**Step4.** 	Day2: record newly developed germinated and 3-point seeds  
**Step5.** 	Continue process on Day3, Day4, and Day5  

**Germination Index**  
<span style="color:green">(5 x gday1 + 4 x gday2… + 1 x gday5)/(5 x n) </span>  
Where gday1 is the number of seed that germinated on day 1  
gday2 is the number of seeds that ADDITIONALLY germinated on day 2 and so on  
n is the total number of seeds plated  

**Sterilization Solution:**  10% Bleach + 0.01% SDS + ddH2O  

**Total Vol.**|200mL   |400mL   | 800mL |                                      	
:---|:---|:---|:---|:---
Bleach | 20mL|	40mL | 80mL | (61.54mL if 1.3x)
2%SDS |	1mL  | 	2mL |4mL|
ddH2O |179mL |358mL |716mL|

--------------
<div style="text-align: right"> [TOC](#TOC) </div>  

### DNA extraction  <a id="DNA_Ext"></a>  
*James Tanaka | 2018.06.06*

**Harvest Tissue**  
**Step 1:** Place two metallic grinding beads in each sample well.  
**Step 2:** At the 2-3 leaf stage of development (approximately 10 days after sowing), cut 2-3 inches of three leaves per sample and place in a deep 96-well plate. OR collect approximately 75mg (3-5in leaf) of fresh tissue into 1.1ml 96 well plates, folding to about ½-3/4in in size.  

>Either sample one plant per sample, or 3-4 plants per sample, depending on the generation and experimental design of the project.  

**Step 3:** Seal the 96-well plate with either 8-well stripe tops (for long term storage) or a securely taped kim wipe (for quick freeze followed by lyophilization).   
**Step 4:** Freeze at -80 °C overnight at minimum. OR freeze in liquid nitrogen and freeze dry for 24-48 hours.  Alternatively, tissue can be harvested and frozen in liquid nitrogen and stored at -80 C until grinding.   

**Grinding DNA**   
**Step 0:** Reserve time on the Geno Grinder by communicating with James Tanaka about project.  
**Step 1:** Turn on the Geno/Grinder 2010 (switch is back left corner).  Allow Geno/Grinder to warm-up for 1min before starting.  
**Step 2:** Adjust the time and rpm settings appropriate for your samples by pushing and turning the knobs.   

|Tissue Type|Prep Type|RPM|Time|
|:--|:--|:--|:--|
|Leaf|lyoph.|1400|1:00|
|Leaf|frozen|1500|:45|
|Root|frozen|1350|:45|
|Crown|frozen|1350|:45|
|Stem|frozen|1350|:45|

> **Going over the recommended RPMs can result in breaking your block, if this happens clean up your mess!** These settings were evaluated for VWR 96-deep well blocks.  

**Step 3:** Unless you are using the freezer blocks, place the orange tray labeled “bottom” flat side down (located ¬¬¬¬drawer below grinder) in the holding tray.  
**Step 4:** Place blocks in orange tray (you MUST have an even number of blocks to even out the machine while running, if you need a height BALANCE, there are some in Lab 411).  
**Step 5:** If you are running more than two blocks the Geno/Grinder can accommodate 4 96-deep well blocks.  Place an orange nesting tray labeled “nesting” on top of the two bottom blocks then place the next two blocks in the nesting tray.  
**Step 6:** Squeeze the button of the twist top lid to bring it down to rest on top of the two blocks.  Finish twisting the knob to tighten lid all the way (finger tight only).  
**Step 7:** Press the green button  
**Step 8:** Wait for the Geno/Grinder to come to a complete stop and the light in green button to become solid again (if you do not wait it can cause an error on the next run).  
**Step 9:** When you are finished turn the Geno/Grinder off.  

**DNA Extraction**  
**Step 0:** Grind to powder on Homogenizer (Genogrinder).  
**Step 1:**  Add 610ul extraction buffer preheated to 65C, mix well and incubate at 65C for 30 min.  Mix samples every 10 mins.  
> Mix well: add a pcr mat on the tubes, mix by inversion + brief centrifuge until dissolved (no compact tissue). If there is compact tissue at the bottom of the tube, it will be okay, you just wont get as much DNA.   

**Step 2:**  Remove from waterbath, wipe off excess water and allow samples to cool to room temperature for about 10 mins, remove steel grinding balls and add 250-300ul 24:1 Chloroform/Isoamyl alcohol  

> Removing balls: pull one strip at a time, being away of broken strips. Hold strip carefully and use a wide magnet (on the side of the hood) to loosen (wiggly) the balls at the bottom of the tubes, and shimmy them up the tube. Usually you can get 6/8. Then go back for other 2.   

**Step 3:**  Shake vigorously to form an emulsion  
**Step 4:**  Centrifuge at top speed for 15 mins (15 min @ 4000rpm or 5 min @ 13000rpm)  
**Step 5:**  If RNA free DNA is desired, add 6ul 5mg/ml RNAse to new tube or plate and proceed, otherwise skip to step 6  
**Step 6:**  Carefully transfer the upper phase to a new tube (approx 500ul; 2 sets of 225; Use 8 channel)  
**Step 7:**  For RNAse treated samples, give a short centrifuge spin (up to 1000rpm) and allow incubation at 37C for 5 minutes, otherwise proceed to step 8  
**Step 8:**  Add 500ul isopropanol and (gently) mix well by inverting 15-25 times (can see DNA form)  
**Step 9:**  For RNAse treated samples, proceed directly to step 10.  Otherwise, optionally let samples sit in -20C for 5-60 mins  
**Step 10:**  Centrifuge at top speed for 30 minutes (30 min @ 4000rpm or 15 min @ 13000rpm)  
**Step 11:**  Carefully discard the liquid, making sure pellet does not dislodge  
**Step 12:**  Add 500ul of 70% Ethanol  
**Step 13:**  Centrifuge at top speed for 15-30 min (15-30 min @ 4000rpm or 5-15 min @ 13000rpm)  
**Step 14:**  Carefully discard liquid, again making sure pellet does not dislodge.  Allow samples to air dry so that no ethanol remains.  This can be done for 20-60 mins at RT, or 15-30 min at 37C.  
**Step 15:**  Resuspend DNA in 50-100ul ddH20 or diluted TE (preferred) and mix on low vortex.  Samples can be refrigerated immediately or allowed to dissolve @ 37C for about 30 mins.    

**Extraction Buffer (100ml):**  

|Final Concentration:|Location | Amount/Stock Concentration| 1 plate 100mL |
|:--|:--|:--|:--|:--|
|ddH2O |by sink| --| 52.8 mL|
|2% CTAB	|dry shelf|  2g/CTAB |2 g|
|1.4M NaCl	|Shelf premade| 28.5ml/5M |28.5 mL|
|20mM EDTA	|Shelf premade| 8ml/0.25M |8 mL|
|100mM Tris-HCl, pH8|Shelf premade | 10ml/1M |10 mL|
|0.2% 2-mercaptoethanol	|-20C Rm 416 | 200ul/2-mercaptoethanol |200 uL|
|0.2mg/ml Pronase E or Proteinase K |-20C Rm 416 |1ml @ 20mg/ml |1 mL|	 

--------------
<div style="text-align: right"> [TOC](#TOC) </div>

### RNA extraction  <a id="RNA_Ext"></a>  
**<span style="color:red">SAFETY PROTOCOL:</span>**  
**Hazards:**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Phenol: Toxic if swallowed, in contact with skin or if inhaled. Causes severe skin burns and eye damage Suspected of causing genetic defects. May cause damage to organs through prolonged or repeated exposure. Toxic to aquatic life. Do not breathe dust/fume/gas/mist/vapors/spray. Wash thoroughly after handling. Do not eat, drink or smoke when using this product. Use only in a well-ventilated area. Avoid release to the environment. If exposed or concerned: Get medical advice/attention. You must sign this training sheet stating you understand the hazards involving the use of phenol.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Chloroform: Hazardous in case of skin contact (irritant), of eye contact (irritant), of ingestion, of inhalation. Slightly hazardous in case of skin contact (permeator). Has CARCINOGENIC and MUTAGENIC effects. Do not ingest. Do not breathe gas/fumes/ vapor/spray. Wear suitable protective clothing. In case of insufficient ventilation, wear suitable respiratory equipment. If ingested, seek medical advice immediately and show the container or the label. Avoid contact with skin and eyes. Keep away from incompatibles such as metals, alkalis. You must sign this training sheet stating you understand the hazards involving the use of chloroform.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Isoamyl Alcohol: Hazardous in case of skin contact (irritant, permeator), of eye contact (irritant), of ingestion, of inhalation. Keep away from heat and any sources of ignition. You must sign this training sheet stating you understand the hazards involving the use of isoamyl alcohol.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Ethanol: Hazardous in case of skin contact (irritant), of eye contact (irritant), of inhalation. Slightly hazardous in case of skin contact (permeator), of ingestion. Flammable, keep away from heat and any sources of ignition. You must sign this training sheet stating you understand the hazards involving the use of ethanol.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Centrifuge: Before using a centrifuge, read the centrifuge safety brochure and sign that you have read it.  

**Protection:**  
- Close-toed shoes are always required when working in the laboratory.  
- Lab coat and RNase free gloves are required at all times during the phenol-chloroform RNA extraction procedure.  
- Wear protective gloves/protective clothing/eye protection/face protection when handling Phenol.  
- When using Isoamyl alcohol & Chloroform: Splash goggles. Lab coat. Vapor respirator. Be sure to use an approved/certified respirator or equivalent. Gloves.   
- When handling liquid nitrogen, always wear protective eye-wear and protective gloves.   

**Waste:**  
Organic waste (all the waste from steps 1- 10) needs to be dumped in a designated organics waste brown glass bottle and disposed of by the official WSU hazardous waste department.  
Phenol and Chloroform waste must be disposed of in accordance with federal, state and local environmental control regulations.   

**Spill Clean-Up:**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Phenol: Exposure to the spilled material may be severely irritating or toxic. Follow personal protective equipment recommendations found in the MSDS. Personal protective equipment needs must be evaluated based on information provided on this sheet and the special circumstances created by the spill including; the material spilled, the quantity of the spill, the area in which the spill occurred, and the expertise of employees in the area responding to the spill. Never exceed any occupational exposure limits. Prevent the spread of any spill to minimize harm to human health and the environment if safe to do so. Wear complete and proper personal protective equipment at a minimum. Dike with suitable absorbent material like granulated clay. Gather and store in a sealed container pending a waste disposal evaluation. Block any potential routes to water systems.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Chloroform: Small Spill: Absorb with an inert material and put the spilled material in an appropriate waste disposal. Large Spill: Absorb with an inert material and put the spilled material in an appropriate waste disposal. Be careful that the product is not present at a concentration level above TLV. Check TLV on the MSDS and with local authorities.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; IsoAmyl Alcohol: Small Spill: Dilute with water and mop up, or absorb with an inert dry material and place in an appropriate waste disposal container. Large Spill: Flammable liquid. Keep away from heat. Keep away from sources of ignition. Stop leak if without risk. Absorb with DRY earth, sand or other non-combustible material. Do not touch spilled material. Prevent entry into sewers, basements or confined areas; dike if needed. Eliminate all ignition sources. Be careful that the product is not present at a concentration level above TLV. Check TLV on the MSDS and with local authorities.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Ethanol: Small Spill: Dilute with water and mop up, or absorb with an inert dry material and place in an appropriate waste disposal container. Large Spill: Flammable liquid. Keep away from heat. Keep away from sources of ignition. Stop leak if without risk. Absorb with DRY earth, sand or other non-combustible material. Do not touch spilled material. Prevent entry into sewers, basements or confined areas; dike if needed. Be careful that the product is not present at a concentration level above TLV. Check TLV on the MSDS and with local authorities.  

**<span style="color:green">PROCEDURE:</span>**  
**DNA-free RNA Isolation For Wheat Seeds (adapted)**  
Isolation of **Total RNA** optimized for wheat seeds in small microfuge tubes.  Protocol is based an Arabidopsis adaption heavily modified to optimize from Sven Nelson based off of a paper published by [Oñate-Sánchez and Vincent-Carbajosa in 2008](http://www.biomedcentral.com/1756-0500/1/93).  For protocol ease, the procedure used for wheat has been left in, and the original protocol with the Arabidopsis notes and more thorough details are found [here](https://github.com/shantel-martinez/Cornell_PHS/blob/master/General%20Research%20Files/Protocols%20and%20Information/SOP%20Phenol-Chloroform%20RNA%20Extraction.v2.docx).      

Follow the <span style="color:blue">blue text comments</span> to help you prep ahead of time and reduce total extraction time.  

*As with any RNA extraction, use extreme caution of RNase contamination.  DEPC-treat any chemicals that you prepare.  For chemicals like Tris that you cannot DEPC-treat, use DEPC’d water to prepare it.  Prepare an RNase-Free zone on your bench where no one will enter and you will never touch with bare hands (<span style="color:blue">do ONCE day before</span>; No need to repeat unless the area has been obviously contaminated or a long time has passed).  You can buy fancy wipes or solutions to wipe this area down, but if you don’t have these, a low percentage bleach solution will work to remove RNases.  Some sources suggest using SDS in water will denature RNases, but this is not recommended.  SDS will partially denature the RNases and then wiping it will spread these still-partially-active RNases around your bench causing more harm than good.  Always wear gloves from a fresh box and use other precautions, but remember also that some of this is overkill and we do it to be extra-safe.  If you happen to touch a tube with your bare hand do not assume that this means that the experiment has failed and toss the tube.  It is likely that one touch will not destroy your sample.  Also, there will be instances where you will have no choice but to use a less-than-RNase-Free zone, such as when using the hood or when storing things in a -20ºC communal freezer.*  

----Day 1----  
**Sample preparation (grinding tissue):**  
*Grinding the tissue is the most important step of the RNA extraction.  You need to get your tissue completely ground without allowing it to defrost.  You do not want to see any tissue start to get soggy.  Keep applying liquid nitrogen as you grind to keep tissue frozen, this requires some care as the liquid nitrogen can cause the sample to splatter up out of the tube or mortar.  Avoid this.*   

**Step 1:** Grind 20mg dry seed in liquid nitrogen and get sample into a 1.5mL microfuge tube (<span style="color:blue">label tubes morning of</span>). For wheat seeds, use both method a. or method b. Method A will get the samples/tissue into a fine powder, which are placed into a cooled microfuge tubes. Then method b will grind the samples more thoroughly with the dremel.    
>a. Use a pre-cooled mortar and pestle (<span style="color:blue">prep day before</span>).  Scrape the ground tissue out and place in an eppendorf tube without allowing it to defrost.  As best as possible, reduce the amount  material lost in transfer from mortar and pestle to tube.    
>
>b. Then grind the seed in the microfuge tube that has been pre-cooled in liquid nitrogen (backflip easy open tubes work well, but don’t seal as tightly).  To pre-cool: Used a metal block with holes for tubes (from a broken tube incubator).  Place this in a styrofoam box and surround with ice (mainly for stability), then flood with liquid nitrogen to cool the block.  Place the open tubes in the block and fill with liquid nitrogen (liqN2) until the bubbling ceases.  This indicates that they have cooled sufficiently (<span style="color:blue">cool right before grinding</span>).  While some liquid nitrogen remains in the tube, carefully add your sample to the tube.  You may wish to cover the top with a metal spatula to prevent the splattering liqN2 from ejecting your samples.  Allow liqN2 to stop bubbling in the tube and when most of the liqN2 has evaporated, grind with a plastic conical tipped head coupled to a handheld dremel (drill-like tool; <span style="color:blue">charge day before </span>).  Wiping the head with 70% EtOH between samples is sufficient to prevent cross-contamination and RNase-degradation.  
>
>*There are commercial products available for this, but a household Dremel will work just as well and cost much less.  When buying a household dremel, a battery powered model is preferred as the cord can get in the way and contaminate your RNase-free area.  In addition to this, I believe the battery powered models tend to be less powerful, the corded models may burn through your tubes.  If you have the option of one with a high-low speed switch this will allow you to optimize the speed if you are worried about burning through your tubes.  Grinding is pretty much instant, no need to hold the grinding head at the bottom for more than a few seconds.  No time for the sample to defrost.   
>**To get optimal yield:** (for Arabidopsis seeds) grind a little longer until you see the sample coat the wall of the tube and it appears that it is “wet.”  Ensure that the tube is still cool, however, you don’t want to heat up the sample sue to friction.  The wetness is most likely the oils from the seed being released.  When you grind this way you will need to scrape some seem material off of the grinding tip after grinding, but it will more than double your yield (600ng/µl → 1200ng/µl).*  

**Step 2:** Immediately add 550µl Extraction Buffer (<span style="color:blue">prep all buffers and aliquot liquids day before </span>) and move to the fume hood to add 550µl chloroform.  Vortex 10 seconds. (**Do not allow sample to defrost before adding the buffer.**)  You may need to stick a pipet tip into the tube to scrape some material off of the edge if you used the optimized grinding method.   
> i. For all vortex steps I have the vortex set at the line between 5 and 6.   
> ii. *Extraction buffer: 0.4M LiCl, 0.2M Tris pH8, 25mM EDTA, 1%SDS
> iii. If you have more than 4 samples for this extraction, then keep the cooled metal block cold.    

**Step 3:** Move vortexed tube (looks like milkshake) to ice if you will have **4 samples or less** to extract; if you will have **more than 4 samples** for this extraction, then place the first tube in the liqN2 cooled metal block that you used for grinding. Then repeat steps 1 and 2 again with the remaining samples.  Place sample #2 and higher in the liqN2 cooled metal block.  The samples in the liqN2 cooled metal block will freeze instantly.  
> Once all samples are ground and in Extraction Buffer and chloroform, proceed to step 4.  **Step 4 through 10 must be done 1 tube at a time.**  You will need to make balances for centrifugation, but if you do even 4 tubes in parallel you are likely to see a decrease in yield from the 1st to the 4th tube.  Leave the remaining tubes in the metal block (add liqN2 periodically to keep cool).   

<span style="color:purple">Take Lunch if 4+ samples are being extracted.</span>  

**Step 4:** Remove the sample that is on ice and vortex briefly to ensure there are no ice chunks present.  Move the next sample in line from the liqN2 metal block to the ice box to defrost (should return to a liquid milkshake by the time you need to use it) and spin the sample 3 min at top speed in a tabletop microcentrifuge (≈13,000 rpm) at RT.  
> Once skilled and familiar with the protocol, you can do 2 samples at a time. So samples #1 and #2 were thawed at the same time, vortexed, and centrifuged together. While sample #3 and #4 are placed on ice to thaw. This is the MAX overlap you should do. Any more may result in error due to time it takes to continue the protocol.  

**Step 5:** Transfer aqueous (top) layer to a fresh tube.  Because you will do two subsequent organic:aqueous separations, you can carefully suck the aqueous layer all the way down to the interphase division formed by the cell debris.  
**Step 6:** (in hood) Add 500µl of water-saturated acidic phenol (it is important not to use pH 6.6 buffered phenol for this, instead use pH 4.5).  Vortex thoroughly after adding phenol.  Add 200µl of chloroform and 8µl of iso-amyl alcohol.  Vortex briefly, but thoroughly. (**This step is important:** if you do not mix thoroughly you will get 3 layers after spinning, if you do you will get 2 layer with a very clear interphase.)  Spin 3 min top speed at RT.   
**Step 7:** VERY CAREFULLY, remove the aqueous (top) layer to a new tube.  **Do this right away, do not let the tube sit.**  Do not suck all the way down to the interphase. (I usually get about 500µl) You want to avoid getting any phenol carry over at this step.  **But**, phenol is very slightly soluble in water, so a small portion of phenol will have gone into the aqueous phase.  In order to remove this we need to do one final chloroform extraction.  
**Step 8:** Add equal volume (for me, 500µl) of chloroform to the tube and vortex thoroughly.  (*Again, **it is important to vortex thoroughly** if you want this step to have any effect.  We want to pull the last little bit of phenol out of the aqueous layer*).  Then… either a. or b. below.  
>a.	For best results, use Phase Lock Gel (PLG) tubes (from 5Prime).  After vortexing, transfer sample with chloroform to a pre-spun PLG tube (<span style="color:blue">spin morning off</span>).  Spin down for 3 min top speed at RT.  
>b.	You can still get great results if you simply spin down the tube at this point (3 min top speed at RT.) (*just be more careful in the next step as noted.*)  

**Step 9:** Transfer supernatant to a fresh tube.  If you have used a PLG tube then you can suck all of the aqueous layer out safely with no worry for organic contamination.  If you did not then just be careful when sucking the top layer off, don’t suck down too close to the interphase and don’t let the tube sit before transferring the aqueous layer as phenol can come out into the top layer if it sits too long (*although unlikely at this point*).  
**Step 10:** Now comes the moment of truth: Add ⅓ volume (for me this was ≈167µl) of 8M LiCl to the tube and invert to mix.  If you get a white precipitant immediately then you have phenol contamination.  **This means you didn’t vortex thoroughly as instructed.** Don’t panic, you can still save your sample, but you will get lower yield.  But if you **do not** see an obvious white precipitant (the white will be obvious, very faint cloudiness or schlieren lines are ok) then you will have phenol-contamination free extractions.  Precipitate at -20ºC for 2 hours minimum.  You will get best yield by precipitating overnight.  **At this point go back to step 4 and repeat steps 4 through 10 until all samples are at -20ºC.  Store for a minimum of 2 hour from last sample.**  

----Day 2----  
**Step 11:** Spin 30 minutes at **4ºC** at 13,000 rpm to recover pellet (<span style="color:blue">cool centri. morning of</span>). A visible brown pellet is a good sign.   
>a.	At the same time, take 10x DNase buffer and from the -20ºC and place it on ice.  The buffer will be frozen and needs to thaw before use.  Place RNase-free DNase I in a stay-cold container (like Stratagene’s StrataCooler Benchtop Coolers) in the freezer ready to take out when you want to use it.  

**Step 12:** Discard the supernatant and dissolve the pellet in 26µl DEPC-treated water.  If pellet is hard to dissolve by flicking the tube (no vortex) then add a 10 minute step at 37ºC to allow it to dissolve more before adding DNase.  
> **If you had a white precipitant in step 11:** After the spin you will see the white precipitant at the bottom with the pellet.  When removing supernatant suck in some liquid and gently “blow” this at the white precipitant.  The phenol will come off in organic bubble droplets fairly easily.  Make sure to remove all of these droplets.  Unfortunately, you will lose some RNA in this step if you do this, but even if you feel like you have harshly blown everything away and your tube looks like it has no pellet, you will still probably get a fair amount of RNA, maybe in the 40-60ng/µl range.  If you are careful, you can still get around 150ng/µl yields consistently this way.  After resuspending pellet look for any (very small) organic droplets floating around.  If they exist, remove them with a 2µl pippetman.  If you are careful, you will still get good quality uncontaminated RNA)  

**Step 13:** For Zymo Research Cat# E1007: Add 6.5µl 10x DNase buffer (<span style="color:blue">thaw on ice morning of</span>) and 3.25µl RNase-free DNase I to each sample (no thaw needed, can use straight from the -20C).  Incubate at 37ºC for 30 minutes.  
> **Important:** Adjust quantities of Buffer and DNase to the recommendation for the brand of your DNase based on a 65µl reaction.   
>This step removes any remaining genomic DNA.  With only 1µl of DNase I, I still saw some genomic DNA contamination on occasion.  

**Step 14:** Add 470µl DEPC-treated water + 7µl **3M NaAc pH5.2** + 250µl **100% EtOH** (fresh bottle, not 95%).  Mix well by inverting.  (NO VORTEX)  Spin 10 min at 4ºC to precipitate carbohydrates.  
**Step 15:** Transfer **supernatant** to new and final tube.  Add 43µl **3M NaAc pH5.2** + 750µl **100% EtOH**.  Mix well by inverting.  Leave at -20ºC for at least 1 hour.    

<span style="color:purple">Lunch Break!!!</span>   

**Step 16:** Spin down 20 min at 4ºC (13,000rpm).  Save pellet.  This is your RNA.  *If you can see a pellet this is a good sign.  If not, it does not mean that you have no RNA.*  
**Step 17:** Wash with 70% EtOH: Add ≈100µl of 70% EtOH and let it wash over the pellet, then suck it up right away and discard.  Do not try to dissolve the pellet in the 70% EtOH.  
**Step 18:** Air-dry open tubes on bench.  (For about 10 minutes, until fairly dry, RNA pellet will not turn white or otherwise change.)  
**Step 19:** Resuspend in 20µl of RNase/DNase-free water (ex: Zymo Cat# W1001-10).  Nanodrop and enjoy! (*It is NOT recommended to use DEPC-treated water at this step.*)  

**Final notes:** 
- Usual yields are > 1000ng/µl from 20mg of dry seed, ≈1500-2000ng/µl if imbibed.  
- To check quality → run semi-denaturing gel electrophoresis of ≈400ng of sample.  
- Even better: Run samples on the bioanalyzer and you can get a RIN score.  

**Original Protocol from:**  
Oñate-Sánchez, L., and Vicente-Carbajosa, J. (2008). DNA-free RNA isolation protocols for 	Arabidopsis thaliana, including seeds and siliques. BMC Res Notes 1, 93.  


**Materials needed:** |	**Keep at:** 
:-----| :----
Extraction buffer: 0.4M LiCl, 0.2M Tris pH8, 25mM EDTA, 1%SDS|	RT
chloroform (does not need to be treated with DEPC)	|4°C
water-saturated acidic phenol (does not need to be treated with DEPC) - pH 4.5	|4°C
8M LiCl	|4°C
100% iso-amyl alcohol	|4°C
10x DNase Buffer	|-20°C
RNase-free DNase	|-20°C
3M NaAc pH5.5	|RT
100% EtOH (fresh bottle, not 95%; keep this RNase-free)	|RT
DEPC-treated water	|RT
70% EtOH	|RT
RNase/DNase-free water - Zymo Research Cat# W1001-10	|RT
Liquid Nitrogen	|-
|
**optional but recommended** Phase Lock Gel tubes (heavy) from 5Prime 1.5mL microcentrifuge tubes	| 
RNase-Free tips (20uL, 200uL, 1000uL)	|  
Ice, metal tube block, Styrofoam box|  	
Centrifuge (one at RT, one at 4°C)	|  
Pipettes (20uL, 200uL, 1000uL)	|  
**optional** handheld dremmel	|   


--------------
<div style="text-align: right"> [TOC](#TOC) </div>  